In [1]:
import pandas as pd
import math
import numpy as np
from prophet import Prophet
import yfinance as yf
import plotly.graph_objects as go
import plotly.express as px 
import random
import matplotlib.pyplot as plt
from XTBClient.api import XTBClient, PERIOD, MODES
import os
from utility import utility
from datetime import datetime, timedelta
from time import sleep
import logging

In [2]:
logging.getLogger('XTBApi.api').setLevel(logging.WARNING)

In [3]:
def store(df, name="bot.csv"):
    """Concats the data from a csv and the data from dataframe, then saves it to the csv"""

    name = os.path.join("csvs", name)
    if os.path.exists(name):
        read = pd.read_csv(name)
        df = pd.concat([df, read])
        # df = df.drop_duplicates(subset=["symbol", df.index])

    df[[col for col in df.columns if col != "index"]].to_csv(name, index=False)
    df.to_csv(name)

    return df

In [4]:
def get_df(name):
    name = os.path.join("csvs", name)
    read = None
    if os.path.exists(name):
        read = pd.read_csv(name)

    return read

In [5]:
def collect():
    c = XTBClient()
    c.login(os.environ.get("XTB_login"), os.environ.get("XTB_pass"))
    ticks = c.get_ticks(["EURUSD"])
    c.logout()

    df = pd.DataFrame(
        columns=["symbol", "ask", "bid", "high", "low", "timestamp", 'date'], 
    ).reset_index(drop=True).set_index('date')
    
    for quot in ticks["quotations"]:
        new = pd.DataFrame(
            [quot]
        )
        
        new['date'] = datetime.fromtimestamp(quot["timestamp"] / 1000)
        new.set_index('date', inplace=True)
        df = pd.concat([df, new])
    return store(df)

In [6]:
# collect()
# df = get_df("bot.csv")

In [7]:
c = XTBClient()
c.login(os.environ.get("XTB_login"), os.environ.get("XTB_pass"))
ticks = c.get_ticks(["EURUSD"])
c.logout()

In [8]:
df = pd.DataFrame(
    columns=["symbol", "ask", "bid", "high", "low", "timestamp", 'date'], 
).reset_index(drop=True).set_index("date")

In [9]:
df.head()

,symbol,ask,bid,high,low,timestamp
date,,,,,,


In [10]:
for quot in ticks["quotations"]:
    new = pd.DataFrame(
        [quot]
    )

    new['date'] = datetime.fromtimestamp(quot["timestamp"] / 1000)
    new.set_index('date', inplace=True)
    df = pd.concat([df, new])
df.head()

,symbol,ask,bid,high,low,timestamp,askVolume,bidVolume,level,exemode,spreadRaw,spreadTable
date,,,,,,,,,,,,
2023-06-13 20:31:17.298,EURUSD,1.07847,1.07839,1.08231,1.07567,1686681077298,350000.0,349000.0,0.0,1.0,0.00008,0.8


In [11]:
name = os.path.join("csvs", "bot.csv")
if os.path.exists(name):
    read = pd.read_csv(name)
    df = pd.concat([df, read])
    # df = df.drop_duplicates(subset=["symbol", df.index])



In [12]:
df.head()

,symbol,ask,bid,high,low,timestamp,askVolume,bidVolume,level,exemode,spreadRaw,spreadTable
date,,,,,,,,,,,,
2023-06-13 20:31:17.298,EURUSD,1.07847,1.07839,1.08231,1.07567,1686681077298,350000.0,349000.0,0.0,1.0,0.00008,0.8


In [13]:
# df[[col for col in df.columns if col != "index"]].to_csv(name, index=False)
df.to_csv(name, index=True)

In [14]:
name = os.path.join("csvs", "bot.csv")
if os.path.exists(name):
    read = pd.read_csv(name, index_col=0)
    df = pd.concat([df, read])
    # df = df.drop_duplicates(subset=["symbol", df.index])

In [15]:
df

,symbol,ask,bid,high,low,timestamp,askVolume,bidVolume,level,exemode,spreadRaw,spreadTable
date,,,,,,,,,,,,
2023-06-13 20:31:17.298000,EURUSD,1.07847,1.07839,1.08231,1.07567,1686681077298,350000.0,349000.0,0.0,1.0,0.00008,0.8
2023-06-13 20:31:17.298,EURUSD,1.07847,1.07839,1.08231,1.07567,1686681077298,350000.0,349000.0,0.0,1.0,0.00008,0.8


In [19]:
c = XTBClient()
c.login(os.environ.get("XTB_login"), os.environ.get("XTB_pass"))
candles = c.get_candles_in_range("EURUSD", PERIOD.ONE_MINUTE.value,  datetime.today() - timedelta(hours=1))
c.logout()

In [26]:
utility.collect_yf("EURUSD=x", period="1y", interval="1h")

[*********************100%***********************]  1 of 1 completed


,datetime,open,high,low,close,adj close
0,2022-06-14 13:00:00,1.043950,1.045697,1.043515,1.044823,1.044823
1,2022-06-14 14:00:00,1.044932,1.047011,1.044823,1.045478,1.045478
2,2022-06-14 15:00:00,1.045369,1.046025,1.043732,1.044168,1.044168
3,2022-06-14 16:00:00,1.044168,1.044823,1.041124,1.041124,1.041124
4,2022-06-14 17:00:00,1.041558,1.042753,1.041016,1.042101,1.042101
...,...,...,...,...,...,...
6205,2023-06-14 09:00:00,1.078865,1.079914,1.078865,1.079214,1.079214
6206,2023-06-14 10:00:00,1.078981,1.080731,1.078981,1.080497,1.080497
6207,2023-06-14 11:00:00,1.080497,1.081198,1.080030,1.080731,1.080731
6208,2023-06-14 12:00:00,1.080614,1.081198,1.080614,1.080964,1.080964


In [2]:
retval = utility.open_transaction(
    MODES.BUY,
    "EURUSD",
    volume=0.02,
    tp=5,
    sl=10
)

2023-06-14 21:08:46 - INFO - XTBApi.api.BaseClient - CMD: login...
2023-06-14 21:08:46 - INFO - XTBApi.api.BaseClient - CMD: get symbol EURUSD...
2023-06-14 21:08:46 - INFO - XTBApi.api.BaseClient - CMD getSymbol with {'command': 'getSymbol', 'arguments': {'symbol': 'EURUSD'}}: done
2023-06-14 21:08:46 - INFO - XTBApi.api.BaseClient - CMD tradeTransaction with {'command': 'tradeTransaction', 'arguments': {'tradeTransInfo': {'cmd': 0, 'comment': '', 'ie_deviation': 0, 'order': 0, 'price': 1.0841, 'sl': 1.0831000000000002, 'symbol': 'EURUSD', 'tp': 1.0846, 'type': 0, 'volume': 0.02}}}: done
2023-06-14 21:08:47 - INFO - XTBApi.api.BaseClient - CMD: logout...


In [3]:
retval

{'order': 518835927}

In [4]:
1.08382*(1.0001)

1.083928382

In [14]:
1.08382 + 1.08382*(0.0002)

1.084036764

In [15]:
price = 1.08382

In [18]:
price + 

1.0829818

In [5]:
c = XTBClient()
c.login(os.environ.get("XTB_login"), os.environ.get("XTB_pass"))
candles = c.transaction_status(retval['order'])
c.logout()
candles

2023-06-14 21:10:42 - INFO - XTBApi.api.BaseClient - CMD: login...
2023-06-14 21:10:42 - INFO - XTBApi.api.BaseClient - CMD tradeTransactionStatus with {'command': 'tradeTransactionStatus', 'arguments': {'order': 518835927}}: done
2023-06-14 21:10:42 - INFO - XTBApi.api.BaseClient - CMD: logout...


{'order': 518835927,
 'requestStatus': 4,
 'message': 'Invalid sl/tp price step',
 'customComment': None,
 'ask': 0.0,
 'bid': 0.0}